In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [4]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [5]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [7]:
# 어제 배운 merge함수이다. outer(외부연결)을 해서 na값을 포함해서 출력하였다.
# 만약 단순하게 merge를 하면 이런 결과가 나온다. 
pd.merge(left, right)
# 디폴트상태는 다음과 같다. 열 이름이 같은것끼리만 찾아서 공통된 데이터에 대한 데이터값을 찾아서 merge를 한 것이다.

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [9]:
# 키 하나만 주고 실행하면 데이터가 6건이 나오게 되는데 key2가 2개로 나눠져서 나오게 되었다. 접미어가 x,,y가 생겨벼렸다. 
pd.merge(left, right, on=["key1"])

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [11]:
pd.merge(left, right, on=["key1"], suffixes=("_left","_right"))
# 접미어를 가져오는 것은 다음과 같이 할 수 있다. 
# 수집된 데이터를 합치는 과정에서 쓰는 merge함수를 알아보았다.
# 잘 다룰 수 있다면 데이터를 원할하게 다룰 수 있다.
# 대규모 프로젝트를 할 경우 필수적인 기술이다.

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [12]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [13]:
right1

,group_val
a,3.5
b,7.0


In [14]:
# 데이터가 같은 것이 없다. 그리고 키가 인덱스로 오는 골 때리는 상황이다.
# value 옆에 group_var을 추가시키고 싶다. 매칭되는 값을 옆에 추가하고 싶다고 하자.
# 간혹 사용하는 경우가 발생한다. a는 연도 b는 뉴스 등 시계열 데이터추가시에 사용한다. 
pd.merge(left1,right1)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [15]:
# 애러 메세지는 공통된 열이 없다고 한다. 그래서 left on과 right on등을 쓰라고 친절하게 설명해준다.
# 이번에는 왼쪽의 키와 오른쪽의 인덱스가 기준이다. 어제는 키와 인덱스가 일치했지만 이번에는 다르다.
pd.merge(left1,right1, left_on='key',right_index=True) 
# 이렇게 가능하다. 근데 허전하다. C는 merge가 안되었다.

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [16]:
pd.merge(left1,right1, left_on='key',right_index=True, how="outer") 
# c도 같이 나오게 되었다. 

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [17]:
# 인덱스가 너저분하다. 인덱스는 필요없을거 같다.
pd.merge(left1,right1, left_on='key',right_index=True, how="outer").reset_index()
# 인덱스가 튀어나왔다. 그런데 옆에 인덱스와 비교해보면 쓸모없어 보인다. 

,index,key,value,group_val
0,0,a,0,3.5
1,2,a,2,3.5
2,3,a,3,3.5
3,1,b,1,7.0
4,4,b,4,7.0
5,5,c,5,NaN


In [18]:
pd.merge(left1,right1, left_on='key',right_index=True, how="outer").reset_index(drop=True)
# 깔끔하게 정리되었다.

,key,value,group_val
0,a,0,3.5
1,a,2,3.5
2,a,3,3.5
3,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [19]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
lefth
# 키가 2개인거 보면 멀티인덱스인 느낌이 난다. 

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [20]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
righth_index
# from_arrays를 기반으로 멀티인덱스를 만들었다. 리스트안에 리스트가 있는 이중 리스트 구조로 설계되었다. 
# 각각의 요소는 튜플형식으로 묶였다.
# 

MultiIndex([('Nevada', 2001),
            ('Nevada', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2001),
            (  'Ohio', 2002)],
           )

In [21]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
righth
# 시리즈 2개도 right 인덱스를 기준으로 지정된 것이다. 위의 멀티인덱스로 데이터프레임을 다중인덱스로 구성한 것이다.

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [22]:
lefth
# 공통된 열이 존재하지는 않는다. 그러나 key1과 key2의 값이 힌트다. 

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [24]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)
# 잘 합쳐졌다. 만약 집나간 네바다를 끌고오고 싶다면 

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [25]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True, how='outer')
# 집나간 네바다도 끌고왔다. 

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [26]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [27]:
right2
# 인덱스가 일치한다. 인덱스를 기준으로 병합을 해보자. 

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [29]:
pd.merge(left2,right2, left_index=True, right_index=True, how='outer')
# 인덱스를 전부 True로 했더니 잘 합쳐진 것을 볼 수 있다. 
# 나이, 연세, age같이 같은 의미지만 다른 인덱스로 된 경우 merge가 가능해진다. 

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [30]:
# merge 말고 join함수도 존재한다.
left2.join(right2)
# 같은 인덱스를 기준으로 합쳐진 것을 볼 수 있다. 
# left2에 있는 것은 전부 나온다. right2에 있는 것은 left2에 있는 요소만 합쳐진다. 

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [31]:
# 여기에 옵션을 줄 수 있다.
left2.join(right2,how ='inner')
# 특이한 점은 inner를 옵션으로 추가하면 결측값이 있으면 전부 제거된다. 

,Ohio,Nevada,Missouri,Alabama
c,3,4,9,10
e,5,6,13,14


In [32]:
left2.join(right2,how ='outer')
# 결측값 포함 전부 나왔다.

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [33]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [34]:
right1

,group_val
a,3.5
b,7.0


In [35]:
left1.join(right1)
# 공통열이 없는 group_bal은 전부 삭제되었다.

,key,value,group_val
0,a,0,NaN
1,b,1,NaN
2,a,2,NaN
3,a,3,NaN
4,b,4,NaN
5,c,5,NaN


In [36]:
left1.join(right1, on = 'key')
# key를 기준으로 배치되었다. 

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [37]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another
# 이중리스트인데 열 인덱스가 2개라서 각각 출력된 것을 볼 수 있다.

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [38]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [39]:
right2
# 이 3개를 전부 합친다고 해보자.

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [40]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [41]:
left2.join(right2, how='outer')
# 이게 좀전에 했던 것이다.

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [42]:
left2.join(right2).join(another)
# 되긴 된다. 3개 연속 연결이 되었다. 

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [43]:
left2.join(right2, how='outer').join(another)
# 꽤 흥미로운 결과가 나왔다. 

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
b,<NA>,<NA>,7,8,NaN,NaN
c,3,4,9,10,9.0,10.0
d,<NA>,<NA>,11,12,NaN,NaN
e,5,6,13,14,11.0,12.0


In [44]:
left2.join([right2,another])
# 결과는 같지만 코드가 짧아지는 결과가 나왔다. 


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [45]:
left2.join([right2,another], how='outer')
# 아우터가 right2에만 된 아까와는 달리 another에도 outer가 적용되었다. 

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


In [46]:
# concat함수로 합치기
# concat은 넘파이. 판다스에 존재하는 합치기 함수이다. 
# 리스트를 합치기보다 붙인다는 느낌이 강하다. 
# 넘파이에서는 np.concatenate라고 쓰고 판다스는 pd.concat로 정의하며 기능은 같다. 
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [47]:
np.concatenate([arr,arr]) # 행으로 합쳐졌다. 

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [48]:
np.concatenate([arr,arr],axis=1) # 열로 합쳐졌다.

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [49]:
arr2 = np.arange(20).reshape((4, 5))
arr2

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [50]:
np.concatenate([arr,arr2])
# 차원이 맞지 않으면 애러가 발생한다. 열 차원의 갯수가 일치하지 않는다. 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4 and the array at index 1 has size 5

In [51]:
np.concatenate([arr,arr2],axis=1)
# 차원이 맞지 않으면 애러가 발생한다. 행 차원의 갯수가 일치하지 않는다. 

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3 and the array at index 1 has size 4

In [52]:
arr3 = np.arange(15).reshape((3, 5))
arr3

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [53]:
np.concatenate([arr,arr3],axis=1)
# 행의 갯수가 동일하기에 합치기가 가능하다. 

array([[ 0,  1,  2,  3,  0,  1,  2,  3,  4],
       [ 4,  5,  6,  7,  5,  6,  7,  8,  9],
       [ 8,  9, 10, 11, 10, 11, 12, 13, 14]])

In [55]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")
s1

a    0
b    1
dtype: Int64

## 

In [56]:
s2

c    2
d    3
e    4
dtype: Int64

In [57]:
s3

f    5
g    6
dtype: Int64

In [59]:
pd.concat([s1,s2])
# 반복 가능한 객체끼리 합치는 것이 가능하다. 인수를 하나로 연결해서(대괄호로 묶어서) 함수실행이 가능하다. 

a    0
b    1
c    2
d    3
e    4
dtype: Int64

In [60]:
pd.concat([s1,s2,s3])
# 위아래로 연결되었다. 

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [62]:
pd.concat([s1,s2,s3],axis = 1)
# 좌우로 연결되긴 했는데 행인덱스가 전부 달라서 모든 자료가 따로논다. axis ='columns'로도 가능하다.

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [63]:
s1

a    0
b    1
dtype: Int64

In [64]:
s3

f    5
g    6
dtype: Int64

In [66]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [67]:
pd.concat([s1,s4])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: Int64

In [68]:
pd.concat([s1,s4],axis=1)

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [69]:
pd.concat([s1,s4],axis=1,join = 'inner')
# 여기서는 how가 아니라 join이다 그리고 inner를 쓰면 na값이 삭제된다. 

,0,1
a,0,0
b,1,1


In [70]:
pd.concat([s1,s4],axis=1,join = 'outer')
# 여기서는 아우터가 디폴트이다. 

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [74]:
res = pd.concat([s1,s2,s3],keys=['one','two','three'])
res
# 인덱스에 이름이 붙었다. 
# 타입은 시리즈이다. 이중 인덱스(멀티인덱스)인 것이다. 
# 어제 배운 것 중 언스택과 스택이 존재했다. 이걸 언스택, 스택을 써보자. 

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: Int64

In [75]:
res.unstack()
# 1차원구조가 2차원구조로 바뀜으로서 데이터프레임이 되었다. 

,a,b,c,d,e,f,g
one,0,1,<NA>,<NA>,<NA>,<NA>,<NA>
two,<NA>,<NA>,2,3,4,<NA>,<NA>
three,<NA>,<NA>,<NA>,<NA>,<NA>,5,6


In [79]:
pd.concat([s1,s2,s3],axis = 1)

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [80]:
pd.concat([s1,s2,s3],axis = 1, keys =['one','two','three'])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [81]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [83]:
df2
# 데이터프레임에 5를 더해서 0123이 5678이 되었다. 꽤 재밌는 방법이다.
# 만약 데이터프레임에 concat을 쓰면 어떨까?

,three,four
a,5,6
c,7,8


In [84]:
pd.concat([df1,df2]) # 

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [85]:
pd.concat([df1,df2],axis =1)# 좌우로 연결되었다.
# 되긴 된다. a와c 인덱스가 존재하기 때문이다. 

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [86]:
df3 = pd.DataFrame(10 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["one", "five"])
df3

,one,five
a,10,11
c,12,13


In [87]:
pd.concat([df1,df3])

,one,two,five
a,0,1.0,NaN
b,2,3.0,NaN
c,4,5.0,NaN
a,10,NaN,11.0
c,12,NaN,13.0


In [88]:
pd.concat([df1,df3], axis = 1) 

,one,two,one,five
a,0,1,10.0,11.0
b,2,3,NaN,NaN
c,4,5,12.0,13.0


In [89]:
pd.concat([df1,df3], axis = 1,keys=['level1','level2']) # 위에 인덱스가 추가되었다.

level1     level2      
     one two    one  five
a      0   1   10.0  11.0
b      2   3    NaN   NaN
c      4   5   12.0  13.0

In [93]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1
# 표준정규분포로 데이터프레임을 만들었다. 

,a,b,c,d
0,0.547279,-1.132391,0.017431,1.446397
1,-0.323293,0.759765,2.091455,-0.156965
2,-0.358873,-1.652112,0.092564,0.681283


In [94]:
df2
# c열과 2행을 제외하면 연결될 것이라고 느껴진다.

,b,d,a
0,0.969656,0.624845,0.656542
1,2.528101,-0.695794,-0.317752


In [96]:
pd.concat([df1,df2])
# 밑에 그대로 붙인 모양새이다. 그리고 인덱스가 보기 안좋다.


,a,b,c,d
0,0.547279,-1.132391,0.017431,1.446397
1,-0.323293,0.759765,2.091455,-0.156965
2,-0.358873,-1.652112,0.092564,0.681283
0,0.656542,0.969656,NaN,0.624845
1,-0.317752,2.528101,NaN,-0.695794


In [98]:
pd.concat([df1,df2]).reset_index(drop=True)
# 깔끔하게 나왔다. 

,a,b,c,d
0,0.547279,-1.132391,0.017431,1.446397
1,-0.323293,0.759765,2.091455,-0.156965
2,-0.358873,-1.652112,0.092564,0.681283
3,0.656542,0.969656,NaN,0.624845
4,-0.317752,2.528101,NaN,-0.695794


In [99]:
# 또다른방법으로는 concat속성을 이용하는 방법이다. 
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,0.547279,-1.132391,0.017431,1.446397
1,-0.323293,0.759765,2.091455,-0.156965
2,-0.358873,-1.652112,0.092564,0.681283
3,0.656542,0.969656,NaN,0.624845
4,-0.317752,2.528101,NaN,-0.695794


In [102]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a
# 결측값이 많다. 

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [103]:
b
# 만약 a가 결측값이면 b데이터를 사용하려고 하고 아니면 a를 그대로 사용하고자 한다.

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [105]:
# 오랜만에 넘파이 웨어함수를 써보자
np.where(pd.isna(a),b,a)
# 배열로 출력이 된다. 그리고 정상적으로 나왔다. 

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [106]:
a.combine_first(b) 
# 이것도 a가 결측값이면 b데이터를 추출한다는 의미이다. 

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [108]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


## 

In [109]:
df2
# df1에 있는 값을 참조해서 df1의 결측값을 df2에 있는 것으로 채우고자 한다. 

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [110]:
df1.combine_first(df2)
# 두 개의 데이터프레임을 합치는 결과가 나왔다. 다만 방식은 약간 다른게 기존의 값의 결측값을 채우면서 칸이 늘어났다.

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [111]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [112]:
data.stack()
# 데이터프레임을 시리즈로 바꾸었다. 언스택을 하면 다시 데이터프레임이 될것이다. 

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [113]:
data.stack().unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [116]:
res = data.stack()
res

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [118]:
res.unstack(level=0)
# 행과 열의 인덱스 순서가 바뀌었다.
# 자연스럽게 행과 열을 바꾸게 되었다. 

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [120]:
res.unstack(level=1)
# 이게 디폴트인 모양이다. 
# 인덱스의 이름을 주는 목적은 인덱스 배열을 구분하기 위함이다. 또한 레벨 입력칸에 'state'나 'number'를 입력하여 입력방법을 늘릴 수 있다.

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [121]:
res.unstack(level='state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [122]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [123]:
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [124]:
s2

c    4
d    5
e    6
dtype: Int64

In [125]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [126]:
data2.unstack().stack()
# 만약 na이도 출력을 하게 하려면

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [127]:
data2.unstack().stack(dropna=False)
# na도 전부 출력되었다.

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [129]:
df = pd.DataFrame({"left": res, "right": res + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [130]:
res

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [131]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [133]:
data2 = data.reindex(columns=['two','three','one'])
data2
# 기존 인덱스 순서를 바꾼 방법이다. 물론 저장해야 반영된다.

number,two,three,one
state,,,
Ohio,1,2,0
Colorado,4,5,3


In [135]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
data

{'도시': ['서울', '서울', '서울', '부산', '부산', '부산', '인천', '인천'],
 '연도': ['2015', '2010', '2005', '2015', '2010', '2005', '2015', '2010'],
 '인구': [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
 '지역': ['수도권', '수도권', '수도권', '경상권', '경상권', '경상권', '수도권', '수도권']}

In [136]:
columns

['도시', '연도', '인구', '지역']

In [137]:
# 위의 자료는 딕셔너리이다. 축이 4개있는 데이터라 4차원 데이터라고 한다.
# 주의할 점은 킷값은 변수가 아니다. 
pd.DataFrame(data)
# 시리즈 4개가 결합해서 데이터프레임이 된 것이다.
# 딥러닝, 머신러닝에서 위의 인덱스를 feature(차원)이라고 한다. 도시, 연도, 인구, 지역등을 말한다.
# 행 데이터를 observation(관측치), instance라고도 한다. 


,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [139]:
cols = ["도시", "연도", "인구", "권역"] # 이름을 바꿔서 써보자.
pd.DataFrame(data,columns=cols)
# 지역이라는 값이 있지 권역이라는 값이 존재하지 않는다. 데이터상에는 권역이라는 값은 존재하지 않는다.

,도시,연도,인구,권역
0,서울,2015,9904312,NaN
1,서울,2010,9631482,NaN
2,서울,2005,9762546,NaN
3,부산,2015,3448737,NaN
4,부산,2010,3393191,NaN
5,부산,2005,3512547,NaN
6,인천,2015,2890451,NaN
7,인천,2010,263203,NaN


In [140]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [141]:
df1.pivot('도시','연도','인구')
# 엑셀의 피봇테이블과 같은 것이다. 
# 피봇과 피봇테이블의 사용목적은 데이터의 구조를 변경하기 위함이다.
# 판다스에서도 테이블의 구조를 변경할 수 있다.
# pivot(행, 열, 데이터) 순으로 배치가 되어있다. 
# 자동으로 정렬까지 해준다. 문자순서대로 정렬되어서 출력한다.
# 엑셀에서도 많이 쓰였듯이 피벗테이블은 판다스에서도 많이 쓰인다. 

C:\Users\user\AppData\Local\Temp\ipykernel_10464\69723028.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df1.pivot('도시','연도','인구')


연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [143]:
df1.set_index('도시')

,연도,인구,지역
도시,,,
서울,2015,9904312,수도권
서울,2010,9631482,수도권
서울,2005,9762546,수도권
부산,2015,3448737,경상권
부산,2010,3393191,경상권
부산,2005,3512547,경상권
인천,2015,2890451,수도권
인천,2010,263203,수도권


In [145]:
df1.set_index(['도시','연도'])

인구   지역
도시 연도                
서울 2015  9904312  수도권
   2010  9631482  수도권
   2005  9762546  수도권
부산 2015  3448737  경상권
   2010  3393191  경상권
   2005  3512547  경상권
인천 2015  2890451  수도권
   2010   263203  수도권

In [146]:
df1.set_index(['도시','연도']).stack()

도시  연도      
서울  2015  인구    9904312
          지역        수도권
    2010  인구    9631482
          지역        수도권
    2005  인구    9762546
          지역        수도권
부산  2015  인구    3448737
          지역        경상권
    2010  인구    3393191
          지역        경상권
    2005  인구    3512547
          지역        경상권
인천  2015  인구    2890451
          지역        수도권
    2010  인구     263203
          지역        수도권
dtype: object

In [148]:
df1.set_index(['도시','연도'])['인구'] 
# 도시연도 데이터에서 인구만 추출한 것이다. 

도시  연도  
서울  2015    9904312
    2010    9631482
    2005    9762546
부산  2015    3448737
    2010    3393191
    2005    3512547
인천  2015    2890451
    2010     263203
Name: 인구, dtype: int64

In [149]:
df1.set_index(['도시','연도'])[['인구']]
# 결과는 동일하지만 대괄호 두개로 쓴 결과 데이터프레임으로 나오게 된다.

인구
도시 연도           
서울 2015  9904312
   2010  9631482
   2005  9762546
부산 2015  3448737
   2010  3393191
   2005  3512547
인천 2015  2890451
   2010   263203

In [150]:
df1.set_index(['도시','연도'])[['인구']].unstack()
# 언스택을 한 결과가 처음에 출력한 피봇과 동일한 것을 볼 수 있을 것이다.

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

In [153]:
df1.pivot(['지역','도시'],'연도','인구')
# 된다. 지역하고 도시를 멀티인덱스로 바꾼 것이다. 

C:\Users\user\AppData\Local\Temp\ipykernel_10464\314631708.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df1.pivot(['지역','도시'],'연도','인구')


연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

In [155]:
df1.pivot('지역','연도','인구')
# 수도권이면서 2010년, 2015년인 것이 2개가 나온다. 그래서 애러가 발생했다.
# 행인덱스와 열인덱스를 지정할 경우 데이터는 1개씩 있어야 한다. 

C:\Users\user\AppData\Local\Temp\ipykernel_10464\2576424078.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  df1.pivot('지역','연도','인구')


ValueError: Index contains duplicate entries, cannot reshape

In [156]:
# 피벗테이블은 피벗함수하고 상당히 비슷하지만 다르다. 
# 피벗테이블은 group by의 성격을 띄고있다. 즉 pivot_table = pivot + groupby이다. 
# groupby함수는 부서별 합계나 평균을 구하는 작업 등, 기준이 되는 열을 그룹화하여 특정연산을 수행하는 경우에 사용한다.
# 그룹화를 하는 목적은 그룹단위로 연산을 수행하기 위함이다. 
# 

In [157]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2
# 데이터 1에 대한 합계와 데이터 2에 대한 합계를 구하고자 한다.
# a의 데이터 1의 합계, b의 데이터 1의 합계.. 이런식으로 4개의 데이터를 얻고 싶다

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [160]:
groups = df2.groupby('key1')
groups
# 내용은 안보인다. 

In [162]:
groups.groups
# 딕셔너리 키가 2개인 구조로 그룹이 되었다.

{'A': [0, 1, 4], 'B': [2, 3]}

In [164]:
groups.get_group('A')
# A그룹의 데이터만 가져왔다. 내가 원하는 열을 기준으로 그룹으로 나눌 수 있고 따로 추출할 수도 있다. 

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
4,A,one,5,50


In [165]:
groups.sum()
# 그룹단위로 합계연산이 수행되었다.
# key2는 숫자가 아니기 때문에 연산이 수행되지 않고 숫자만 연산이 되었다.
# 우리가 원하는 결과가 출력되었다.

C:\Users\user\AppData\Local\Temp\ipykernel_10464\1171057664.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  groups.sum()


,data1,data2
key1,,
A,8,80
B,7,70


In [167]:
df2
# 특정열 하나. df2의 data1열에 대해서만 그룹화를 하고싶다고 하자. 그룹화 기준은 key1열을 기준으로 한다. 

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [169]:
df2.data1

0    1
1    2
2    3
3    4
4    5
Name: data1, dtype: int64

In [170]:
df2.data1.groupby(df2.key1).sum()
# 일부에 대한 합계가 출력되었다. 

key1
A    8
B    7
Name: data1, dtype: int64

In [171]:
df2.groupby('key1').sum()
# 바로 원하는 결과가 나왔다. 옆에 ['data1']을 붙여도 위와 똑같은 결과를 얻을 수 있다. 

C:\Users\user\AppData\Local\Temp\ipykernel_10464\3163032127.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df2.groupby('key1').sum()


,data1,data2
key1,,
A,8,80
B,7,70


In [172]:
df2.groupby('key1')['data1'].sum()
# 메모리 측면에서 더 효율적이다. 데이터1,2를 다 더한 전 코드보다는 데이터 1만 추출해서 더했기에 메모리 효율이 더 좋다.

key1
A    8
B    7
Name: data1, dtype: int64

In [178]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [179]:
df2.data1.groupby([df2.key1,df2.key2]).sum()
#key1과 key2 그룹으로 두 가지 연산을 나열하였다.

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [180]:
# 문제 
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [211]:
df2.data1.groupby([df2['key1'],df2['key2']]).sum().unstack()

key2,one,two
key1,,
A,6,2
B,3,4


In [199]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [209]:
df1.인구.groupby([df1["지역"],df1["연도"]]).sum().unstack()

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


In [212]:
# 복습시간.
# 연습을 해보기가 필요하다. 데이터를 도출하는 법을 돌아보자.
# 오전에 merge함수를 공부했었다. 
df1 = pd.DataFrame({
    '고객번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    '이름': ['둘리', '도우너', '또치', '길동', '희동', '마이콜', '영희']
}, columns=['고객번호', '이름'])
df1

,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


In [213]:
df2 = pd.DataFrame({
    '고객번호': [1001, 1001, 1005, 1006, 1008, 1001],
    '금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns=['고객번호', '금액'])
df2

,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


In [214]:
# 고객번호가 같은것끼리 연결해서 데이터프레임을 형성하고 싶었다고 하자.
pd.merge(df1,df2) 
# 고객번호를 기준으로 합쳐졌다.
# 여기서 전체를 합친다고 해보자. 

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


In [215]:
pd.merge(df1,df2 , how='outer') 
#전체출력이 되었다.

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


In [216]:
pd.merge(df1,df2,how='right')
# 고객번호를 기준으로 출력되었고 df2의 기록이 전체 다 나왔다. 즉 df2 기준으로 합쳐졌다.

,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1005,희동,15000
3,1006,마이콜,5000
4,1008,NaN,100000
5,1001,둘리,30000


In [238]:
df1 = pd.DataFrame({
    '품종': ['setosa', 'setosa', 'virginica', 'virginica'],
    '꽃잎길이': [1.4, 1.3, 1.5, 1.3]},
    columns=['품종', '꽃잎길이'])
df1

,품종,꽃잎길이
0,setosa,1.4
1,setosa,1.3
2,virginica,1.5
3,virginica,1.3


In [239]:
df2 = pd.DataFrame({
    '품종': ['setosa', 'virginica', 'virginica', 'versicolor'],
    '꽃잎너비': [0.4, 0.3, 0.5, 0.3]},
    columns=['품종', '꽃잎너비'])
df2

,품종,꽃잎너비
0,setosa,0.4
1,virginica,0.3
2,virginica,0.5
3,versicolor,0.3


In [ ]:
# 아래와 같은 출력을 원한다고 하자. 
  품종	     꽃잎길이	꽃잎너비
0	setosa	    1.4	      0.4
1	setosa	    1.3	      0.4
2	virginica	1.5	      0.3
3	virginica	1.5	      0.5
4	virginica	1.3	      0.3
5	virginica	1.3	      0.5

In [220]:
pd.merge(df1,df2)

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


In [232]:
df1 = pd.DataFrame({
    '고객명': ['춘향', '춘향', '몽룡'],
    '날짜': ['2018-01-01', '2018-01-02', '2018-01-01'],
    '데이터': ['20000', '30000', '100000']})
df1

,고객명,날짜,데이터
0,춘향,2018-01-01,20000
1,춘향,2018-01-02,30000
2,몽룡,2018-01-01,100000


In [233]:
df2 = pd.DataFrame({
    '고객명': ['춘향', '몽룡'],
    '데이터': ['여자', '남자']})
df2

,고객명,데이터
0,춘향,여자
1,몽룡,남자


In [230]:
pd.merge(df1,df2)
# 결합이 안되는 데이터였다. 만약 열 이름인 '데이터'를 성별로 바꾸면 정상적으로 나올 것이다. 
# df1의 데이터는 숫자이다. df2의 데이터는 성별이다. 아예 다른 수치이다. 

,고객명,날짜,데이터


In [231]:
df2 = pd.DataFrame({
    '고객명': ['춘향', '몽룡'],
    '성별': ['여자', '남자']})
pd.merge(df1,df2)
# 열 이름이 같지 않기 때문에 합쳐지지 않은 것이다. 

,고객명,날짜,데이터,성별
0,춘향,2018-01-01,20000,여자
1,춘향,2018-01-02,30000,여자
2,몽룡,2018-01-01,100000,남자


In [234]:
# 만약 고객명, 데이터라는 이름으로 병합을 하기 위해서는 어떻게 해야할까? 
# 지금은 이름이 같기 때문에 합쳐지지 않는다. 그래서 어떤 열로 merge하라는 지시를 해야한다. 
# on은 이럴때 쓰는 함수이다. 
pd.merge(df1,df2,on='고객명')
# 이런 경우가 간혹 발생한다. 

,고객명,날짜,데이터_x,데이터_y
0,춘향,2018-01-01,20000,여자
1,춘향,2018-01-02,30000,여자
2,몽룡,2018-01-01,100000,남자


In [235]:
# 코딩은 긴 시간이 필요하다. 코딩은 어려울 수 밖에 없다.
# 코딩을 모르겠다면 보고 따라하는 것도 좋은 방법이다.
# 코드를 작성하는 것은 안보고 작성할 수 있지만 시간이 많이 필요하다.
# 반복하고 기초부터 알아가는 것만으로도 많은 도움이 될 것이다.

In [240]:
pd.merge(df1,df2) 
# 이런 사례가 상당히 많다. 

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


In [241]:
df1 = pd.DataFrame({
    '이름': ['영희', '철수', '철수'],
    '성적': [1, 2, 3]})
df1
#사실 데이터프레임을 만들 일은 많지 않다. 데이터프레임이 만들어진 상태에서 오는 경우가 많다.

,이름,성적
0,영희,1
1,철수,2
2,철수,3


In [242]:
df2 = pd.DataFrame({
    '성명': ['영희', '철수', '철수'],
    '점수': [1, 2, 3]})
df2

,성명,점수
0,영희,1
1,철수,2
2,철수,3


In [245]:
df.merge(df1,df2)
# 될리가 없다. 이름, 성명은 다르고 성적, 점수는 다르다. 

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [246]:
pd.merge(df1,df2, left_on='이름',right_on='성명')
# 이런 사례가 상당히 많다. 열 이름이 달라서 발생하는 문제가 많은 것이다.

,이름,성적,성명,점수
0,영희,1,영희,1
1,철수,2,철수,2
2,철수,2,철수,3
3,철수,3,철수,2
4,철수,3,철수,3


In [247]:
df1 = pd.DataFrame({
    '품종': ['setosa', 'setosa', 'virginica', 'virginica'],
    '꽃잎길이': [1.4, 1.3, 1.5, 1.3]},
    columns=['품종', '꽃잎길이'])
df1

,품종,꽃잎길이
0,setosa,1.4
1,setosa,1.3
2,virginica,1.5
3,virginica,1.3


In [248]:
df2 = pd.DataFrame({
    '품종': ['setosa', 'virginica', 'virginica', 'versicolor'],
    '꽃잎너비': [0.4, 0.3, 0.5, 0.3]},
    columns=['품종', '꽃잎너비'])
df2

,품종,꽃잎너비
0,setosa,0.4
1,virginica,0.3
2,virginica,0.5
3,versicolor,0.3


In [249]:
# 사실 여기는 on을 안써도 품종이라는 열이 중복되기에 on을 쓰든 안쓰든 결과가 같다.
pd.merge(df1,df2, on = "품종")

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


In [250]:
df1 = pd.DataFrame({
    '도시': ['서울', '서울', '서울', '부산', '부산'],
    '연도': [2000, 2005, 2010, 2000, 2005],
    '인구': [9853972, 9762546, 9631482, 3655437, 3512547]})
df1

,도시,연도,인구
0,서울,2000,9853972
1,서울,2005,9762546
2,서울,2010,9631482
3,부산,2000,3655437
4,부산,2005,3512547


In [251]:
df2 = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[['부산', '부산', '서울', '서울', '서울', '서울'],
           [2000, 2005, 2000, 2005, 2010, 2015]],
    columns=['데이터1', '데이터2'])
df2

데이터1  데이터2
부산 2000     0     1
   2005     2     3
서울 2000     4     5
   2005     6     7
   2010     8     9
   2015    10    11

In [252]:
# 흔치는 않는 사례다. 데이터프레임을 보아하니 df2는 멀티인덱스이다.
# df1과 df2를 합칠 것이다. 도시를 기준으로 연결하고 싶다. 
# 다만 여기의 데이터는 연도도 존재한다. 인구열이 옆에 추가되었으면 좋겠다. 
# 즉 df2의 도시, 연도와 df1열의 도시, 연도를 합쳐서 써야한다는 뜻이다.
pd.merge(df1,df2, left_on=['도시','연도'],right_index=True)
# 열은 on, 행은 index를 사용하는 것을 기억해두자.

,도시,연도,인구,데이터1,데이터2
0,서울,2000,9853972,4,5
1,서울,2005,9762546,6,7
2,서울,2010,9631482,8,9
3,부산,2000,3655437,0,1
4,부산,2005,3512547,2,3


In [253]:
df1 = pd.DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]],
    index=['a', 'c', 'e'],
    columns=['서울', '부산'])
df1

,서울,부산
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [254]:
df2 = pd.DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'],
    columns=['대구', '광주'])
df2

,대구,광주
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [268]:
pd.concat([df1,df2], axis = 1,sort=True)
# sort는 생각도 못했다... 이거 코드 쓴 사람 천잰가

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [260]:
pd.merge(df1,df2, left_index=True,right_index=True, how='outer')

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [263]:
df1.join(df2, how='outer')

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0
